## Pré-carregamento de dados

Os dados originais estão em format "Fixed Width", o que exigirá um pré-processamento de forma a obter as informações desejadas.

Para isso, nos aproveitaremos do script de input do SAS, que lista a posição inicial e o tamanho de cada uma das variáveis.

In [11]:
import pandas as pd
import re
import unidecode
import pickle

In [9]:
# parseando o script de input do SAS e obtendo as informações desejadas da posição das variáveis

codes_desc = {'colspecs': [], 'descs': [], 'codes': []}
code2desc = {}

with open('./aux_files/input_vars_PES2015.txt') as f:
    for l in f:
        c = l.rstrip().split()

        #colspec
        pos = int(c[0][1:])
        size = int(c[2].replace('.', '').replace('$', ''))
        codes_desc['colspecs'].append((pos - 1, pos - 1 + size))
        
        # names
        desc = unidecode.unidecode('_'.join(c[3:])[2:-2]).lower().replace(',','')
        codes_desc['descs'].append(desc)
        
        #codes
        codes_desc['codes'].append(c[1])
        
        #
        code2desc[c[1]] = desc

codes_desc_df = pd.DataFrame(codes_desc)

In [3]:
# lendo o arquivo em Fixed Width 
#    - V9906 e V9907 são variavéis categoricas que podem com 0, por isso forçamos o formato str

data = pd.read_fwf('./dados/PES2015.txt', colspecs=codes_desc['colspecs'], names=codes_desc['codes'], dtype={'V9906': str, 'V9907': str})

In [4]:
# salvando em csv para acesso mais fácil na sequência do projeto

data.to_csv("./dados/pes2015_data.csv", index=False, encoding='utf-8')

In [5]:
# salvando uma alternativa de 25 pct aleatorio da base para manipulação mais fácil em exploração

data.sample(frac=0.25).to_csv("./dados/pes2015_data_sample.csv", index=False, encoding='utf-8')

In [13]:
# salvando o mapa de code -> desc

pickle.dump(code2desc, open("./dados/code2desc.p", "wb"))

<hr>